In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'classification_monthly_dataset.parquet'

In [ ]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"Shape: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

In [ ]:
# Info general
print("=" * 60)
print("INFO GENERAL")
print("=" * 60)
print(f"\nTotal registros: {len(df):,}")
print(f"Período: {df['anio'].min()} - {df['anio'].max()}")
print(f"Municipios: {df['codigo_municipio'].nunique()}")
print(f"\nValores nulos:")
nulls = df.isnull().sum()
print(nulls[nulls > 0])

## 1. Balance de Clases - Nivel de Riesgo

In [ ]:
# Balance de clases: nivel_riesgo
print("=" * 60)
print("BALANCE DE CLASES: nivel_riesgo")
print("=" * 60)

riesgo_counts = df['nivel_riesgo'].value_counts()
riesgo_pct = df['nivel_riesgo'].value_counts(normalize=True) * 100

balance_riesgo = pd.DataFrame({
    'count': riesgo_counts,
    'porcentaje': riesgo_pct.round(2)
})
# Ordenar por nivel lógico
order = ['BAJO', 'MEDIO', 'ALTO']
balance_riesgo = balance_riesgo.reindex(order)
print(balance_riesgo)

# Ratio de desbalance
ratio = riesgo_counts.max() / riesgo_counts.min()
print(f"\nRatio max/min: {ratio:.2f}")

In [ ]:
# Visualización nivel_riesgo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'BAJO': 'green', 'MEDIO': 'orange', 'ALTO': 'red'}
order = ['BAJO', 'MEDIO', 'ALTO']

# Barplot
ax1 = axes[0]
riesgo_ordered = riesgo_counts.reindex(order)
bars = ax1.bar(order, riesgo_ordered.values, color=[colors[x] for x in order])
ax1.set_xlabel('Nivel de Riesgo')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de nivel_riesgo')
for bar, val in zip(bars, riesgo_ordered.values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 100, f'{val:,}', ha='center')

# Pie chart
ax2 = axes[1]
ax2.pie(riesgo_ordered.values, labels=order, autopct='%1.1f%%', 
        colors=[colors[x] for x in order])
ax2.set_title('Proporción de nivel_riesgo')

plt.tight_layout()
plt.show()

## 2. Balance de Clases - Incremento Delitos

In [ ]:
# Balance de clases: incremento_delitos
print("=" * 60)
print("BALANCE DE CLASES: incremento_delitos")
print("=" * 60)

incremento_counts = df['incremento_delitos'].value_counts()
incremento_pct = df['incremento_delitos'].value_counts(normalize=True) * 100

balance_incremento = pd.DataFrame({
    'count': incremento_counts,
    'porcentaje': incremento_pct.round(2)
})
balance_incremento.index = balance_incremento.index.map({0: 'Sin incremento (0)', 1: 'Con incremento (1)'})
print(balance_incremento)

# Ratio de desbalance
ratio = incremento_counts.max() / incremento_counts.min()
print(f"\nRatio max/min: {ratio:.2f}")

In [ ]:
# Visualización incremento_delitos
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

labels = ['Sin incremento', 'Con incremento']
colors = ['steelblue', 'coral']

# Barplot
ax1 = axes[0]
bars = ax1.bar(labels, incremento_counts.values, color=colors)
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de incremento_delitos')
for bar, val in zip(bars, incremento_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 100, f'{val:,}', ha='center')

# Pie chart
ax2 = axes[1]
ax2.pie(incremento_counts.values, labels=labels, autopct='%1.1f%%', colors=colors)
ax2.set_title('Proporción de incremento_delitos')

plt.tight_layout()
plt.show()

## 3. Análisis de Features por Target

In [ ]:
# Distribución de total_delitos por nivel_riesgo
fig, ax = plt.subplots(figsize=(10, 6))

order = ['BAJO', 'MEDIO', 'ALTO']
palette = {'BAJO': 'green', 'MEDIO': 'orange', 'ALTO': 'red'}

sns.boxplot(data=df, x='nivel_riesgo', y='total_delitos', order=order, palette=palette, ax=ax)
ax.set_title('Distribución de total_delitos por nivel_riesgo')
ax.set_xlabel('Nivel de Riesgo')
ax.set_ylabel('Total Delitos')

plt.tight_layout()
plt.show()

# Estadísticas por grupo
print("\nEstadísticas de total_delitos por nivel_riesgo:")
print(df.groupby('nivel_riesgo')['total_delitos'].describe().round(2))

In [ ]:
# Análisis temporal por nivel_riesgo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Por año
riesgo_por_anio = df.groupby(['anio', 'nivel_riesgo']).size().unstack(fill_value=0)
riesgo_por_anio = riesgo_por_anio[['BAJO', 'MEDIO', 'ALTO']]
riesgo_por_anio.plot(kind='bar', stacked=True, ax=axes[0], color=['green', 'orange', 'red'])
axes[0].set_title('Nivel de Riesgo por Año')
axes[0].set_xlabel('Año')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Nivel Riesgo')

# Por mes
riesgo_por_mes = df.groupby(['mes', 'nivel_riesgo']).size().unstack(fill_value=0)
riesgo_por_mes = riesgo_por_mes[['BAJO', 'MEDIO', 'ALTO']]
riesgo_por_mes.plot(kind='bar', stacked=True, ax=axes[1], color=['green', 'orange', 'red'])
axes[1].set_title('Nivel de Riesgo por Mes')
axes[1].set_xlabel('Mes')
axes[1].legend(title='Nivel Riesgo')

plt.tight_layout()
plt.show()

## 4. Correlaciones

In [ ]:
# Seleccionar features numéricas relevantes
feature_cols = ['total_delitos', 'poblacion_total', 'densidad_poblacional', 
                'n_centros_poblados', 'area_km2', 'anio', 'mes']

# Agregar columnas de delitos si existen
delito_cols = [c for c in df.columns if c.startswith('tasa_') or c in 
               ['ABIGEATO', 'AMENAZAS', 'DELITOS SEXUALES', 'EXTORSION', 'HOMICIDIOS', 'HURTOS', 'LESIONES', 'VIOLENCIA INTRAFAMILIAR']]
feature_cols.extend([c for c in delito_cols if c in df.columns][:8])

# Matriz de correlación
fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = df[feature_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, ax=ax, annot_kws={'size': 8})
ax.set_title('Matriz de Correlación')
plt.tight_layout()
plt.show()

## 5. Conclusiones y Recomendaciones

In [ ]:
print("=" * 60)
print("CONCLUSIONES")
print("=" * 60)

# Análisis nivel_riesgo
ratio_riesgo = riesgo_counts.max() / riesgo_counts.min()
print(f"\nnivel_riesgo:")
print(f"  Clases: {list(riesgo_counts.index)}")
print(f"  Ratio max/min: {ratio_riesgo:.2f}")
if ratio_riesgo < 1.5:
    print(f"  Status: ✅ MUY BALANCEADO - Ideal para clasificación")
else:
    print(f"  Status: ⚡ Leve desbalance - Considerar class_weight")

# Análisis incremento_delitos
ratio_incremento = incremento_counts.max() / incremento_counts.min()
print(f"\nincremento_delitos:")
print(f"  Clases: 0 (sin incremento), 1 (con incremento)")
print(f"  Ratio max/min: {ratio_incremento:.2f}")
if ratio_incremento < 2:
    print(f"  Status: ✅ BALANCEADO - No requiere ajustes")
else:
    print(f"  Status: ⚡ MODERADO - Usar class_weight='balanced'")

print("\n" + "=" * 60)
print("RECOMENDACIONES PARA MODELO")
print("=" * 60)
print("""
1. nivel_riesgo (multiclase):
   - Dataset bien balanceado (~33% cada clase)
   - Modelos: XGBoost, LightGBM, Random Forest
   - Métrica: F1-macro, accuracy

2. incremento_delitos (binario):
   - Leve desbalance (58/42%)
   - Usar class_weight='balanced'
   - Modelos: Logistic Regression, XGBoost
   - Métrica: F1, AUC-ROC

3. General:
   - Validación: StratifiedKFold(n_splits=5)
   - Features temporales son importantes (anio, mes)
   - Considerar lag features de meses anteriores
""")